<a href="https://colab.research.google.com/github/lrcherubini/photoIndexer/blob/main/PhotoIndexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Área Compartilhada

Bloco 1: Instalação e Imports
Esta é a primeira célula do seu notebook.

In [13]:
# Célula 1: Instalações e Imports
!pip install -q google-generativeai pandas

import google.generativeai as genai
import pandas as pd
import pathlib
import json
import numpy as np
import time
from google.colab import drive # Importa a biblioteca para o Google Drive

Bloco 2: Conexão e API Key
Rode esta célula. Ela vai pedir permissão para acessar seu Google Drive e solicitar sua API Key do Gemini.

In [14]:
# Célula 2: Conexão com Google Drive e API
from google.colab import userdata

# 1. Montar o Google Drive
#    (Vai abrir uma janela pop-up para você dar permissão)
drive.mount('/content/drive', force_remount=True)

# 2. Configurar a API Key do Gemini
#    (Recomendado: Salve sua key como um "Secret" no Colab com o nome 'PhotoIndexer')
try:
    api_key = userdata.get('PhotoIndexer')
    genai.configure(api_key=api_key)
    print("Google Drive e API Key configurados com sucesso!")
except Exception as e:
    print(f"Erro ao configurar a API Key. Verifique se você criou o Secret 'PhotoIndexer'.\n{e}")

Mounted at /content/drive
Google Drive e API Key configurados com sucesso!


# INDEXAÇÃO (Gerar Legendas e Embeddings)

Objetivo: Rodar uma única vez sobre sua pasta de fotos. Ele vai:

Ler todas as fotos do seu Google Drive.

Para cada foto, gerar a legenda (alt-text).

Para cada foto, gerar o embedding (vetor de busca).

Salvar tudo em um único arquivo indice_fotos.csv no seu Drive.

Bloco 3: Painel de Controle (Seus Parâmetros)
Aqui você define suas pastas e modelos. Edite as variáveis aqui para que correspondam à sua organização no Google Drive.

In [11]:
# Célula 3: Painel de Controle (EDITE AQUI)

# 1. Onde estão suas fotos no Google Drive?
#    (Ex: "/content/drive/MyDrive/MeuSite/Fotografias")
PASTA_DAS_FOTOS = "/content/drive/MyDrive/Férias 2016/Rafting" # <- MUDE AQUI

# 2. Onde você quer salvar o arquivo de índice?
#    (Recomendo salvar na mesma pasta)
ARQUIVO_DE_SAIDA = "/content/drive/MyDrive/Férias 2016/Rafting/indice_fotos.csv" # <- MUDE AQUI

# 3. Modelos de IA
MODELO_VISAO = 'models/gemini-2.0-flash'      # Para gerar legendas
MODELO_EMBEDDING = 'models/embedding-001'     # Para gerar vetores de busca

# 4. Prompt para a legenda (Objetivo 1)
PROMPT_LEGENDA = """
Sua única tarefa é analisar a imagem e retornar um objeto JSON.

VOCÊ É UM FOTÓGRAFO PROFISSIONAL E ESPECIALISTA EM ACESSIBILIDADE.
Seu objetivo é catalogar esta imagem para um portfólio de fotografia, considerando também a futura busca baseada em texto descritivo.

O JSON deve ter a seguinte estrutura:
{
  "caption": "Uma legenda descritiva e literal para fins de acessibilidade (alt-text). Foque no sujeito principal, ação e cenário.",
  "keywords": {
    "subjects": ["palavra1", "palavra2", "palavra3"],  // Sujeito principal da foto (pessoa, objeto, cenário)
    "techniques": ["palavra4", "palavra5", "palavra6"],  // Aspectos técnicos ou artísticos da foto (ex: 'luz natural', 'composição')
    "mood": ["palavra7"]  // Sentimento ou atmosfera da imagem (opcional)
  },
  "category": "categoria_da_imagem",  // Ex: 'Retrato', 'Paisagem', 'Foto de Ação', 'Cenas Urbanas', etc.
  "context": "Breve descrição do que está acontecendo na imagem e o que ela transmite. Foque no contexto da cena e nas emoções ou ações."
}

Responda apenas com o JSON, sem adicionar texto adicional. Sempre responda em português BR.
"""

print(f"Arquivos de fotos serão lidos de: {PASTA_DAS_FOTOS}")
print(f"O índice será salvo em: {ARQUIVO_DE_SAIDA}")

Arquivos de fotos serão lidos de: /content/drive/MyDrive/Férias 2016/Rafting
O índice será salvo em: /content/drive/MyDrive/Férias 2016/Rafting/indice_fotos.csv


Bloco 4: O Script Principal (Indexador)
Este é o bloco mais importante. Ele vai demorar para rodar se você tiver muitas fotos (pois faz 2 chamadas de API por foto e espera 1 segundo).

In [12]:
# Célula 4: Script de Indexação (Versão 3.0 - Lendo JSON)
import json # Certifique-se de que o json está importado

# 1. Carregar os modelos
model_visao = genai.GenerativeModel(MODELO_VISAO)
model_embedding = genai.GenerativeModel(MODELO_EMBEDDING)

# 2. Encontrar as fotos
pasta = pathlib.Path(PASTA_DAS_FOTOS)
arquivos_fotos = list(pasta.glob('*.jpg')) + list(pasta.glob('*.png')) + list(pasta.glob('*.jpeg')) + list(pasta.glob('*.JPG')) + list(pasta.glob('*.PNG')) + list(pasta.glob('*.JPEG'))

print(f"Encontradas {len(arquivos_fotos)} fotos. Iniciando indexação...")

# 3. Loop de processamento
resultados_finais = []
for caminho_foto in arquivos_fotos:
    arquivo_upload = None  # Inicia a variável como nula
    try:
        print(f"Processando: {caminho_foto.name}...")

        # Faz o upload da imagem para a API
        arquivo_upload = genai.upload_file(path=caminho_foto)
        print(f"{caminho_foto.name} uploaded.")

        # --- OBJETIVO 1: Gerar JSON Estruturado ---
        response_legenda_json = model_visao.generate_content([PROMPT_LEGENDA, arquivo_upload])

        # Variáveis padrão em caso de falha no parse
        caption = "ERRO: Falha no parse"
        keywords_subjects = ""
        keywords_techniques = ""
        keywords_mood = ""
        category = ""
        context = ""
        try:
            # Limpar a resposta do modelo (às vezes vem com ```json ... ```)
            texto_limpo = response_legenda_json.text.strip().replace("```json", "").replace("```", "")
            dados_json = json.loads(texto_limpo)

            caption = dados_json.get('caption', 'ERRO: Missing caption')
            category = dados_json.get('category', '')
            context = dados_json.get('context', '')

            # Parsear o dicionário aninhado de keywords
            keywords_dict = dados_json.get('keywords', {})
            keywords_subjects = ", ".join(keywords_dict.get('subjects', []))
            keywords_techniques = ", ".join(keywords_dict.get('techniques', []))
            keywords_mood = ", ".join(keywords_dict.get('mood', []))

        except Exception as e:
            print(f"--- Aviso: Falha ao parsear JSON da legenda. Erro: {e}")
            print(f"--- Resposta Bruta: {response_legenda_json.text}")
            caption = response_legenda_json.text.strip() # Salva o texto bruto em caso de falha

        print(f"  Legenda: {caption}")
        print(f"  Categoria: {category}")
        print(f"  Contexto: {context}")
        print(f"  Keywords (Assuntos): {keywords_subjects}")
        print(f"  Keywords (Técnica): {keywords_techniques}")
        print(f"  Keywords (Sentimento): {keywords_mood}")

        # Converter o array de keywords em uma string simples (separada por vírgula) para o CSV
        keywords_str = ", ".join(keywords)

        print(f"Legenda: {legenda}")
        print(f"Keywords: {keywords_str}")

        # --- OBJETIVO 2 (Parte A): Gerar Embedding (DO TEXTO) ---

        # Criar um "super-texto" combinando todos os campos para um embedding rico
        texto_para_embedding = f"""
        Legenda: {caption}
        Categoria: {category}
        Contexto: {context}
        Assuntos: {keywords_subjects}
        Técnicas: {keywords_techniques}
        Sentimento: {keywords_mood}
        """

        response_embedding = genai.embed_content(
            model=MODELO_EMBEDDING,
            content=texto_para_embedding, # Usando o novo "super-texto"
            task_type="RETRIEVAL_DOCUMENT"
        )
        embedding = response_embedding['embedding']

        # Converter a lista de embedding em um texto JSON para salvar no CSV
        embedding_json = json.dumps(embedding)
        print(f"  Embedding do texto gerado.")


        # Guardar resultados
        resultados_finais.append({
            "arquivo": caminho_foto.name,
            "legenda": caption,
            "categoria": category,
            "contexto": context,
            "keywords_assuntos": keywords_subjects,
            "keywords_tecnica": keywords_techniques,
            "keywords_sentimento": keywords_mood,
            "embedding_json": embedding_json
        })

        # Pausa para evitar limite da API
        time.sleep(1)

    except Exception as e:
        # Se algo der errado, imprima o erro e continue para a próxima foto
        print(f"!!! Erro ao processar {caminho_foto.name}: {e}")
        # O bloco 'finally' cuidará da limpeza

    finally:
        # Bloco de limpeza: será executado SEMPRE
        if arquivo_upload:
            try:
                genai.delete_file(arquivo_upload.name)
            except Exception as e:
                print(f"--- Aviso: Falha ao limpar o arquivo {arquivo_upload.name}. {e}")

# 4. Salvar tudo em um arquivo CSV
df = pd.DataFrame(resultados_finais)
df.to_csv(ARQUIVO_DE_SAIDA, index=False)

print("\n--- SUCESSO! ---")
print(f"Processamento concluído. {len(df)} fotos foram indexadas.")
print(f"Resultados salvos em: {ARQUIVO_DE_SAIDA}")
print(f"\nSeu CSV agora contém as colunas: {df.columns.to_list()}")

Encontradas 72 fotos. Iniciando indexação...
Processando: 21-11-16  13 h 001.JPG...
21-11-16  13 h 001.JPG uploaded.
  Legenda: Um jovem sorrindo e dando joinha dentro de um caiaque inflável azul e preto com um colete salva-vidas preto e um capacete azul com estrelas, remando em um rio calmo com uma pá amarela.
  Categoria: Foto de Ação
  Contexto: A imagem captura um momento de alegria e aventura enquanto um jovem rema em um rio calmo, demonstrando entusiasmo pela atividade ao dar um joinha. O cenário tranquilo e ensolarado contribui para uma atmosfera positiva e convidativa.
  Keywords (Assuntos): caiaque, rio, menino, remo, esporte aquático
  Keywords (Técnica): fotografia de natureza, luz natural
  Keywords (Sentimento): alegre, aventura
Legenda: Grupo de pessoas sorrindo em bote de rafting no rio.
Keywords: rafting, rio, pessoas, esporte, aventura
  Embedding do texto gerado.
Processando: 21-11-16  13 h 002.JPG...
21-11-16  13 h 002.JPG uploaded.
  Legenda: Seis pessoas sorrindo e

# BUSCA (Encontrar Fotos por Depoimento)

Bloco 3: carrega seu índice na memória e prepara o modelo de IA. Você só precisa rodar uma vez por sessão

In [19]:
# Célula 3: Carregar o Índice de Fotos e o Modelo

# --- EDITE AQUI ---
# 1. Onde está seu arquivo de índice? (O mesmo que o Notebook 1 criou)
ARQUIVO_DE_INDICE = "/content/drive/MyDrive/Férias 2016/Rafting/indice_fotos.csv" # <- MUDE AQUI
# --- FIM DA EDIÇÃO ---

# 2. Carregar o modelo de embedding (para texto)
model_embedding = genai.GenerativeModel('models/embedding-001')

# 3. Carregar o índice de fotos do CSV
try:
    df_fotos = pd.read_csv(ARQUIVO_DE_INDICE)
    if df_fotos.empty:
        print(f"Erro: O arquivo de índice '{ARQUIVO_DE_INDICE}' está vazio.")
    else:
        # Pré-processar os embeddings (converter JSON de volta para vetor)
        # Fazemos isso uma vez na carga para ser rápido
        df_fotos['embedding'] = df_fotos['embedding_json'].apply(lambda x: np.array(json.loads(x)))
        print(f"Índice de {len(df_fotos)} fotos carregado e processado com sucesso.")

except FileNotFoundError:
    print(f"Erro: Arquivo de índice '{ARQUIVO_DE_INDICE}' não encontrado.")
except Exception as e:
    print(f"Erro ao ler o arquivo de índice: {e}")

Índice de 72 fotos carregado e processado com sucesso.


O Script de Busca (Seu Painel Principal). Alterar o MEU_DEPOIMENTO e rodar a célula para ter seus resultados.

In [21]:
# Célula 4: Script de Busca Semântica (Objetivo 2 - Parte B)

# --- EDITE AQUI ---
# 1. Qual texto você quer usar para buscar?
MEU_DEPOIMENTO = "Adorei o atendimento, me senti muito acolhido e feliz com o resultado."
# --- FIM DA EDIÇÃO ---


if 'df_fotos' not in locals() or df_fotos.empty:
    print("Erro: O índice de fotos não foi carregado. Rode a Célula 3 primeiro.")
else:
    print(f"Gerando embedding para o depoimento: '{MEU_DEPOIMENTO}'")

    # 2. Gerar o embedding para o depoimento (a "query")
    response_query = genai.embed_content(
        model=MODELO_EMBEDDING,
        content=MEU_DEPOIMENTO,
        task_type="RETRIEVAL_QUERY" # Importante: "RETRIEVAL_QUERY" para busca
    )
    query_embedding = np.array(response_query['embedding'])

    # 3. Calcular a similaridade (Produto Escalar)
    #    (Compara o vetor do depoimento com CADA vetor de texto da foto)
    df_fotos['similaridade'] = df_fotos['embedding'].apply(
        lambda texto_emb: np.dot(query_embedding, texto_emb)
    )

    # 4. Ordenar e mostrar os melhores resultados
    df_resultados = df_fotos.sort_values(by='similaridade', ascending=False)

    print("\n--- 🏆 MELHORES FOTOS ENCONTRADAS 🏆 ---")
    print(f"Para o depoimento: '{MEU_DEPOIMENTO}'\n")

    # 5. Mostrar as 5 fotos mais relevantes de forma legível
    for index, row in df_resultados.head(5).iterrows():
        print(f"--- Posição #{index+1} (Similaridade: {row['similaridade']:.4f}) ---")
        print(f"Arquivo: {row['arquivo']}")
        print(f"Legenda: {row['legenda']}")
        print(f"Contexto: {row['contexto']}")
        print(f"Keywords (Assuntos): {row['keywords_assuntos']}")
        print("\n")

Gerando embedding para o depoimento: 'Adorei o atendimento, me senti muito acolhido e feliz com o resultado.'

--- 🏆 MELHORES FOTOS ENCONTRADAS 🏆 ---
Para o depoimento: 'Adorei o atendimento, me senti muito acolhido e feliz com o resultado.'

--- Posição #11 (Similaridade: 0.6117) ---
Arquivo: 21-11-16  13 h 012.JPG
Legenda: Um grupo de pessoas em um bote inflável azul e preto, remando em corredeiras de um rio, vestindo coletes salva-vidas e capacetes amarelos, com o instrutor em pé na parte de trás do bote.
Contexto: Um grupo de pessoas aproveita um dia de rafting, remando com entusiasmo através das corredeiras. O ambiente é de aventura e diversão, com todos colaborando para navegar pelo rio.
Keywords (Assuntos): rafting, rio, corredeiras, bote inflável, pessoas, coletes salva-vidas, capacetes, remo


--- Posição #19 (Similaridade: 0.6075) ---
Arquivo: 21-11-16  13 h 020.JPG
Legenda: Uma mulher sorridente de capacete amarelo e colete salva-vidas azul pula de uma rocha para o rio com o